# See data_frame_credits_maker.py for script that recreates this ipython notebook. 

In [1]:
import pandas as pd
import sqlite3
import ast
import itertools
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [2]:
def get_buildings():
    buildings_all = pd.read_csv("original_data/PublicLEEDProjectDirectory.csv", skiprows=3, encoding="ISO-8859-1")
    buildings_all = buildings_all[buildings_all['Isconfidential'] == 'No']
    buildings_2009 = buildings_all[buildings_all["LEEDSystemVersionDisplayName"] == "LEED-NC v2009"]
    return(buildings_2009)

In [29]:
def my_filter_2009(df, country_code):
    countries = df.dropna(subset = ['PointsAchieved'])
    countries['PointsAchieved'] = countries['PointsAchieved'].convert_objects(convert_numeric=True)
    countries = countries[countries['PointsAchieved'] > 39]
    countries = countries[countries['Country'] == country_code]
    countires = countries[countries["OwnerTypes"] == 'Corporate: Privately Held']
    return(countries)

In [4]:
def read_table(sqlite_data):
# Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(sqlite_data)
    credits = pd.read_sql_query("SELECT * from points_data", con)
    con.close()
    return(credits)

In [5]:
def make_df(credits):
    count = 0
    x_df = pd.DataFrame()
    for i in credits.iloc[:,0]:
        x_df[i] = pd.Series({key :value[1] for i in ast.literal_eval(credits.iloc[count,1]) for key, value in i.items()})
        count +=1
    new_x = x_df.transpose()
    return(new_x)

In [55]:
def make_merge(country, credits_df):
#     country.merge(credits_df, how='inner', left_on='ID', left_index=False, right_index=True)
#     country.set_index(['ID'])
    merged = country.merge(credits_df, right_index=True, left_on=['ID'])                 
#     merged = credits_df.join(country.set_index(['ID']), on = ['ID'], how = 'left')
    return(merged)
# df_train_csv.join(df_train_fly.set_index(['Date']), on = ['Date'], how = 'right', lsuffix='_x')

In [56]:
def making_it_happen(sqlite_data, country_code):
    buildings_2009 = get_buildings()
    country = my_filter_2009(buildings_2009, country_code)
    credits = read_table(sqlite_data)
    credits_df = make_df(credits)
    final_df = make_merge(country, credits_df)
    return(final_df)

In [57]:
turkey_df = making_it_happen("data/turkey_scraped.sqlite3", 'TR')

/Users/trippshealy/Gdrive/research/leed_countries/.direnv/python-3.4.3/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/trippshealy/Gdrive/research/leed_countries/.direnv/python-3.4.3/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [71]:
print(turkey_df.columns)

Index(['ID', 'Isconfidential', 'ProjectName', 'Street', 'City', 'State',
       'Zipcode', 'Country', 'LEEDSystemVersionDisplayName', 'PointsAchieved',
       'CertLevel', 'CertDate', 'IsCertified', 'OwnerTypes', 'GrossSqFoot',
       'TotalPropArea', 'ProjectTypes', 'OwnerOrganization',
       'RegistrationDate', 'EAc1', 'EAc2', 'EAc3', 'EAc4', 'EAc5', 'EAc6',
       'EQc1', 'EQc2', 'EQc3.1', 'EQc3.2', 'EQc4.1', 'EQc4.2', 'EQc4.3',
       'EQc4.4', 'EQc5', 'EQc6.1', 'EQc6.2', 'EQc7.1', 'EQc7.2', 'EQc8.1',
       'EQc8.2', 'IDc1', 'IDc2', 'MRc1.1', 'MRc1.2', 'MRc2', 'MRc3', 'MRc4',
       'MRc5', 'MRc6', 'MRc7', 'SSc1', 'SSc2', 'SSc3', 'SSc4.1', 'SSc4.2',
       'SSc4.3', 'SSc4.4', 'SSc5.1', 'SSc5.2', 'SSc6.1', 'SSc6.2', 'SSc7.1',
       'SSc7.2', 'SSc8', 'WEc1', 'WEc2', 'WEc3'],
      dtype='object')
